In [1]:
from datetime import datetime, timezone

import codebook
import numpy as np
import pandas as pd
import scipy.stats as scs
import seaborn as sns
from IPython.display import display
from matplotlib import pyplot as plt

pd.options.display.max_rows = 500
pd.options.display.max_columns = 200

labels = codebook.variable_labels()
encodings = codebook.student_encodings()

In [4]:
rdf = pd.read_csv("no_weights_hsls_17_student_pets_sr_v1_0.csv")
rdf.head()

,Unnamed: 0,STU_ID,SCH_ID,X1NCESID,X2NCESID,STRAT_ID,PSU,X2UNIV1,X2UNIV2A,X2UNIV2B,X3UNIV1,X4UNIV1,X1SEX,X1RACE,X1HISPANIC,X1WHITE,X1BLACK,X1ASIAN,X1PACISLE,X1AMINDIAN,X1HISPTYPE,X1ASIANTYPE,X1NATIVELANG,X1DUALLANG,X1STDOB,X1TXMTH,X1TXMSEM,X1TXMSCR,X1TXMTSCOR,X1TXMQUINT,X1TXMPROF1,X1TXMPROF2,X1TXMPROF3,X1TXMPROF4,X1TXMPROF5,X1MACC,X1PARRESP,X1P1RELATION,X1PAR1EDU,X1PAR1EMP,X1PAR1OCC2,X1PAR1OCC6,X1PAR1OCC_STEM1,X1PAR1OCC_STEM2,X1PAR1RACE,X1P2RELATION,X1PAR2EDU,X1PAR2EMP,X1PAR2OCC2,X1PAR2OCC6,X1PAR2OCC_STEM1,X1PAR2OCC_STEM2,X1PAR2RACE,X1PAREDU,X1PARPATTERN,X1MOMRESP,X1MOMREL,X1MOMEDU,X1MOMEMP,X1MOMOCC2,X1MOMOCC6,X1MOMOCC_STEM1,X1MOMOCC_STEM2,X1MOMRACE,X1DADRESP,X1DADREL,X1DADEDU,X1DADEMP,X1DADOCC2,X1DADOCC6,X1DADOCC_STEM1,X1DADOCC_STEM2,X1DADRACE,X1HHNUMBER,X1FAMINCOME,X1POVERTY,X1POVERTY130,X1POVERTY185,X1SES,X1SESQ5,X1SES_U,X1SESQ5_U,X1MTHID,X1MTHUTI,X1MTHEFF,X1MTHINT,X1SCIID,X1SCIUTI,X1SCIEFF,X1SCIINT,X1SCHOOLBEL,X1SCHOOLENG,X1STU30OCC2,X1STU30OCC6,X1STU30OCC_STEM1,X1STU30OCC_STEM2,X1STUEDEXPCT,X1PAREDEXPCT,X1STUPRVSCHL_R,X1IEPFLAG,...,X2PAR1EMP_IM,X2PAR2EMP_IM,X2PAR1OCC_IM,X2PAR2OCC_IM,X2MOMREL_IM,X2MOMEDU_IM,X2MOMEMP_IM,X2MOMOCC_IM,X2DADREL_IM,X2DADEDU_IM,X2DADEMP_IM,X2DADOCC_IM,X2HHNUMB_IM,X2FAMINC_IM,X2POVERTY_IM,X2SES_IM,X2STUEDEX_IM,X2PAREDEX_IM,X3CLASSES_IM,X3WORK_IM,X3HSCRED_IM,X3HSCREDTY_IM,X3LASTHSD_IM,X3DROPSTAT_IM,X3EVERDROP_IM,X3CLGANDW_IM,X4X2SES1,X4X2SES2,X4X2SES3,X4X2SES4,X4X2SES5,X4X2SES1_U,X4X2SES2_U,X4X2SES3_U,X4X2SES4_U,X4X2SES5_U,X4X2SES_IM,X4HSCOMPDATE_IM,X4HSCOMPSTAT_IM,X4FB16ENRSTAT_IM,X4HS2PSMOS_IM,X4PSLFSTFB16_IM,X4ATNDCLG16FB_IM,X4EVRATNDCLG_IM,X4CHILDREN_IM,X4EMPHRSFB16_IM,X4INCOMECAT_IM,X4ANYJOB_IM,X4UNEMP16FB_IM,X4WORKING16FB_IM,X5PFYTFEDWRK_IM,X5PFYSTATEAMT_IM,X5PFYSTATNEED_IM,X5PFYSTATNOND_IM,X5PFYSTGTAMT_IM,X5PFYSTMILAMT_IM,X5PFYSTNDMRT_IM,X5PFYSTNDONLY_IM,X5PFYSTNOND1_IM,X5PFYSTVETAMT_IM,X5PFYSTWKAMT_IM,X5PFYSTMERIT_IM,X5PFYSTLNAMT_IM,X5PFYVOCHELP_IM,X5PFYINSTAMT_IM,X5PFYINGRTAMT_IM,X5PFYINSTNEED_IM,X5PFYINSMERIT_IM,X5PFYINATHAMT_IM,X5PFYINSTNOND1_IM,X5PFYEMPLWAIV_IM,X5PFYINSMILAMT_IM,X5PFYINLNAMT_IM,X5PFYINSWAIV_IM,X5PFYINSTWRK_IM,X5PFYINSTVETAMT_IM,X5PFYMERITAID_IM,X5PFYSEOGAMT_IM,X5PFYTITIVAMT_IM,X5PFYNEEDAID_IM,X5PFYTITIVAIDREC_IM,X5PFYSTAIDREC_IM,X5PFYINSTAIDREC_IM,X5PFYANYAIDREC_IM,X5PFYCAMPAMT_IM,X5PFYFEDNEED_IM,X5PFYFEDPACK_IM,X5PFYT4GRTAMT_IM,X5PFYNETPRICEALL_IM,X5PFYTOTAID2_IM,X5PFYNETPRICEGRT_IM,X5PFYPELLPACK_IM,X5PFYTOTLOAN_IM,X5PFYTOTLOAN2_IM,X5PFYTOTLOAN3_IM,X5EVRFEDAPP_IM,X5FEDAPP14_IM,X5FEDAPP15_IM,X5FEDAPP16_IM,X5PFYTUITION_IM
0,0,10001,-5,-5,-5,-5,-5,11,1,1,1111,11111,1,8,0,1,0,-5,-5,-5,-5,-5,-5,1,199502,0.8304,0.2549,50.4919,59.3710,5,0.9997,0.9669,0.8976,0.3126,0.1013,0,1,2,5,4,21,-5,0,-5,8,1,5,4,29,-5,4,-5,8,5,1,0,1,5,4,29,-5,4,-5,8,1,1,5,4,21,-5,0,-5,8,3,10,0,0,0,1.5644,5,1.6907,5,1.76,1.31,0.95,0.12,0.91,-0.33,1.10,-0.23,0.84,-1.41,25,-5,0,-5,8,6,-5,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.5184,1.5184,1.5184,1.5184,1.5184,1.8774,1.8774,1.8774,1.8774,1.8774,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,10002,-5,-5,-5,-5,-5,11,1,1,1111,11111,2,8,0,1,0,-5,-5,-5,-5,-5,-5,1,199511,-0.2956,0.2696,35.8045,47.6821,2,0.9714,0.5204,0.1768,0.0582,0.0512,-8,1,1,3,3,43,-5,0,-5,5,2,2,2,51,-5,0,-5,8,3,1,1,1,3,3,43,-5,0,-5,5,0,1,2,2,51,-5,0,-5,8,6,3,0,0,1,-0.3699,2,-0.3923,2,-0.57,1.31,0.55,-9.00,0.91,1.69,0.69,-9.00,0.05,-0.20,29,-5,4,-5,11,6,-5,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.1124,-0.1124,-0.1124,-0.1124,-0.1124,-0.0754,-0.0754,-0.0754,-0.0754,-0.0754,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8
2,2,10003,-5,-5,-5,-5,-5,11,1,1,1111,11111,2,3,0,0,1,-5,-5,-5,-5,-5,-5,1,199506,1.2997,0.2555,56.0477,64.2431,5,1.0000,0.9914,0.9838,0.6606,0.2001,0,1,1,7,4,21,-5,0,-5,3,-7,-7,-7,-7,-5,-7,-5,-7,7,7,1,1,7,4,21,-5,0,-5,3,0,0,0,0,0,-

In [9]:
df = rdf.copy()

columns = [
    "STU_ID",
    "X1RACE",
    "X1WHITE",
    "X1SEX",
    "X1SESQ5",
    "X4PS1LEVEL",
    "X3REGION",
    "X1LOCALE",
    "P1MTHCOMP",
    "P1SCICOMP",
    "S1MTCHMFDIFF",
    "S1STCHMFDIFF",
    "S1MTHCOMP",
    "S1SCICOMP",
    "X1TXMQUINT",
    "S1MPERSON1",
    "S2MPERSON1",
    "S4MPERSON1",
    "S1SPERSON1",
    "S2SPERSON1",
    "S4SPERSON1",
    "S1MUNDERST",
    "S1SUNDERST",
    "X5STEM1GPA",
    "S3FIELD_STEM",
    "X4ENTMJST",
]

cat_columns = [c for c in columns if c in encodings.keys()]

df = df[columns]

for column in df.columns:
    if column in encodings:
        df[column] = codebook.to_categorical(df[column])

df.pipe(codebook.relabel).head()

,Student ID,X1 Student's race/ethnicity-composite,X1 Student is White-composite,X1 Student's sex,X1 Quintile coding of X1SES composite,X4 First postsecondary institution level,X3 School region,X1 School locale (urbanicity),P1 E05A Comparison of females' and males' abilities in math,P1 E05B Comparison of females' and males' abilities in science,S1 C11H 9th grader's fall 2009 math teacher treats males/females differently,S1 D11H 9th grader's fall 2009 science teacher treats males/females differently,S1 E14B How 9th grader compares males and females in math,S1 E14C How 9th grader compares males and females in science,X1 Mathematics quintile score,S1 C01A 9th grader sees himself/herself as a math person,S2 E01A Teenager sees himself/herself as a math person,S4 B24A Sees self as a math person,S1 D01A 9th grader sees himself/herself as a science person,S2 E03A Teenager sees himself/herself as a science person,S4 B24C Sees self as a science person,S1 C02 How often 9th grader thinks he/she really understands math assignments,S1 D02 How often 9th grader thinks he/she really understands science assignments,Postsecondary Transcript: STEM (using SMART grant definition of STEM): known GPA,S3 C05C Major will be considering - STEM code,"X4 Major considering upon postsecondary entry is in a science, technology, engineering, or math (STEM) field"
0,10001,"White, non-Hispanic",Student is White,Male,Fifth quintile (highest),2-year,Midwest,Rural,Males are somewhat better,Males are somewhat better,Disagree,Disagree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Strongly agree,Disagree,Agree,Agree,Agree,Strongly agree,Often,Often,3.7,No,NaN
1,10002,"White, non-Hispanic",Student is White,Female,Second quintile,4-year,Northeast,Rural,Males are much better,Females and males are the same,Strongly disagree,Strongly disagree,Males are somewhat better,Females and males are the same,Second quintile,Disagree,Disagree,Agree,Agree,Disagree,Agree,Sometimes,Often,-7.0,No,No
2,10003,"Black/African-American, non-Hispanic",Student is not White,Female,Fifth quintile (highest),4-year,West,Suburb,Males are much better,Males are somewhat better,Strongly disagree,Strongly disagree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Disagree,Strongly disagree,Strongly agree,Disagree,Strongly disagree,Strongly agree,Often,Sometimes,2.7,No,Yes
3,10004,"White, non-Hispanic",Student is White,Female,Fifth quintile (highest),NaN,South,Suburb,Males are much better,Females are much better,Disagree,Disagree,Females and males are the same,Females and males are the same,Third (middle) quintile,Agree,NaN,NaN,Disagree,NaN,NaN,Sometimes,Sometimes,-6.0,NaN,NaN
4,10005,"White, non-Hispanic",Student is White,Male,Fourth quintile,NaN,South,City,Females and males are the same,Females and males are the same,Strongly disagree,Agree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Strongly agree,NaN,Disagree,Disagree,Strongly agree,Disagree,Often,Often,-6.0,No,NaN


Below is the variables included in the data exploration.

In [10]:
# This is for updating the variable list in Mplus input files
print("\n".join(columns))

STU_ID
X1RACE
X1WHITE
X1SEX
X1SESQ5
X4PS1LEVEL
X3REGION
X1LOCALE
P1MTHCOMP
P1SCICOMP
S1MTCHMFDIFF
S1STCHMFDIFF
S1MTHCOMP
S1SCICOMP
X1TXMQUINT
S1MPERSON1
S2MPERSON1
S4MPERSON1
S1SPERSON1
S2SPERSON1
S4SPERSON1
S1MUNDERST
S1SUNDERST
X5STEM1GPA
S3FIELD_STEM
X4ENTMJST


Select columns of interest.

**For comparison between students who were included vs. excluded**

In [11]:
df.shape

(23503, 26)

In [23]:
df.isna().apply(any, axis=1).value_counts()

True     18831
False     4672
dtype: int64

In [17]:
df.assign(**{c: df[c].cat.codes for c in columns if c in encodings})

,STU_ID,X1RACE,X1WHITE,X1SEX,X1SESQ5,X4PS1LEVEL,X3REGION,X1LOCALE,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA,S3FIELD_STEM,X4ENTMJST
0,10001,7,1,1,4,1,1,3,3,3,1,1,2,2,4,3,1,2,2,2,3,3,3,3.7,0,-1
1,10002,7,1,0,1,0,0,3,4,2,0,0,3,2,1,1,1,2,2,1,2,2,3,-7.0,0,0
2,10003,2,0,0,4,0,3,1,4,3,0,0,2,2,4,1,0,3,1,0,3,3,2,2.7,0,1
3,10004,7,1,0,4,-1,2,1,4,0,1,1,2,2,2,2,-1,-1,1,-1,-1,2,2,-6.0,-1,-1
4,10005,7,1,1,3,-1,2,0,2,2,0,2,2,2,4,3,-1,1,1,3,1,3,3,-6.0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23498,35202,4,1,0,2,-1,2,1,-1,-1,0,0,2,2,3,2,2,-1,3,2,-1,2,3,-8.0,-1,-1
23499,35203,4,1,0,0,-1,0,1,-1,-1,0,0,0,2,1,3,1,2,0,3,2,3,0,-6.0,0,-1
23500,35204,7,1,0,2,0,1,0,2,2,1,1,0,0,2,2,1,1,1,2,2,3,2,0.0,0,0
23501,35205,7,1,1,4,1,1,1,2,2,0,0,0,0,4,0,0,2,1,0,1,2,2,-7.0,0,0


In [24]:
df["X5STEM1GPA"]

0        3.7
1       -7.0
2        2.7
3       -6.0
4       -6.0
        ... 
23498   -8.0
23499   -6.0
23500    0.0
23501   -7.0
23502    4.0
Name: X5STEM1GPA, Length: 23503, dtype: float64

In [34]:
# Run this cell when comparing students included vs. excluded

df["INCLUDED"] = (df >= 0).apply(all, axis=1)

In [35]:
df.shape

(23503, 27)

In [36]:
df.head()

,STU_ID,X1RACE,X1WHITE,X1SEX,X1SESQ5,X4PS1LEVEL,X3REGION,X1LOCALE,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA,S3FIELD_STEM,X4ENTMJST,INCLUDED
0,10001,8,1,1,5,2,2,4,4,4,3,3,3,3,5,1,3,2,2,2,1,4,4,3.7,0,-9,False
1,10002,8,1,2,2,1,1,4,5,3,4,4,4,3,2,3,3,2,2,3,2,3,4,-7.0,0,0,False
2,10003,3,0,2,5,1,4,2,5,4,4,4,3,3,5,3,4,1,3,4,1,4,3,2.7,0,1,True
3,10004,8,1,2,5,-7,3,2,5,1,3,3,3,3,3,2,-8,-4,3,-8,-4,3,3,-6.0,-8,-7,False
4,10005,8,1,1,4,-7,3,1,3,3,4,2,3,3,5,1,-9,3,3,1,3,4,4,-6.0,0,-7,False


In [37]:
df.shape

(23503, 27)

In [38]:
df["INCLUDED"].value_counts()

False    19901
True      3602
Name: INCLUDED, dtype: int64

# Chi-square

In [39]:
race = df[df["X1RACE"] != -9]
race = race[race["X1RACE"] != 4]

In [40]:
race = race.groupby(["X1RACE", "INCLUDED"]).size().to_frame("count")
race = race.reset_index()
race

,X1RACE,INCLUDED,count
0,1,False,149
1,1,True,16
2,2,False,1614
3,2,True,338
4,3,False,2179
5,3,True,271
6,5,False,2982
7,5,True,393
8,6,False,1657
9,6,True,284


In [41]:
race_pivot = race.reset_index().pivot(
    columns="X1RACE", index="INCLUDED", values="count"
)
race_pivot

X1RACE,1,2,3,5,6,7,8
INCLUDED,,,,,,,
False,149,1614,2179,2982,1657,95,9797
True,16,338,271,393,284,15,2285


In [42]:
pd.crosstab(
    race["INCLUDED"],
    race["X1RACE"],
    race["count"],
    aggfunc=sum,
    normalize="index",
)

X1RACE,1,2,3,5,6,7,8
INCLUDED,,,,,,,
False,0.008066,0.087371,0.117956,0.161425,0.089698,0.005143,0.530342
True,0.004442,0.093837,0.075236,0.109106,0.078845,0.004164,0.634370


In [44]:
pd.crosstab(
    race["INCLUDED"],
    race["X1RACE"],
    race["count"],
    aggfunc=sum,
    normalize="columns",
)

X1RACE,1,2,3,5,6,7,8
INCLUDED,,,,,,,
False,0.90303,0.826844,0.889388,0.883556,0.853684,0.863636,0.810876
True,0.09697,0.173156,0.110612,0.116444,0.146316,0.136364,0.189124


In [43]:
pd.crosstab(
    df["INCLUDED"],
    df["X1RACE"],
)

X1RACE,-9,1,2,3,4,5,6,7,8
INCLUDED,,,,,,,,,
False,1006,149,1614,2179,422,2982,1657,95,9797
True,0,16,338,271,0,393,284,15,2285


In [48]:
pd.crosstab(df["INCLUDED"], df["X1SEX"])

X1SEX,-9,1,2
INCLUDED,,,
False,6,10377,9518
True,0,1596,2006


In [47]:
scs.chi2_contingency(
    pd.crosstab(
        df["INCLUDED"],
        df["X1SEX"],
    ),
    correction=False,
)

(76.30686300188728,
 2.6926210303439965e-17,
 2,
 array([[5.08045781e+00, 1.01380536e+04, 9.75786597e+03],
        [9.19542186e-01, 1.83494643e+03, 1.76613403e+03]]))

In [45]:
scs.chi2_contingency(race_pivot, correction=False)

(174.46930944246174,
 5.067281011424522e-35,
 6,
 array([[  138.07678369,  1633.49019253,  2050.23103058,  2824.29784824,
          1624.28507361,    92.05118913, 10110.56788222],
        [   26.92321631,   318.50980747,   399.76896942,   550.70215176,
           316.71492639,    17.94881087,  1971.43211778]]))

In [70]:
# I could do this in the following way, but don't know how to not use correction.
table = [[63, 312], [128, 68]]
chi2, pval, dof, expected = scs.chi2_contingency(table)
print(chi2, pval, dof, expected)

133.8800822184013 5.803715994221986e-31 1 [[125.43782837 249.56217163]
 [ 65.56217163 130.43782837]]


In [44]:
# This is how you have to select a column when looking at just students included in the analysis

# df[df["INCLUDED"]]["X1SEX"].value_counts()

2    2006
1    1596
Name: X1SEX, dtype: int64

In [8]:
df = df[(df >= 0).apply(all, axis=1)]

In [9]:
df.shape

(3602, 26)

In [10]:
df["X1SEX"].value_counts().sort_index()

# 1 = Male, 2 = Female

1    1596
2    2006
Name: X1SEX, dtype: int64

In [11]:
df["S1MUNDERST"].value_counts().sort_index()

# 1 = Never, 2 = Rarely, 3 = Sometimes, 4 = Often

1      16
2     141
3     975
4    2470
Name: S1MUNDERST, dtype: int64

In [12]:
df["S1MPERSON1"].value_counts().sort_index()

# 1 = Strongly agree, 2 = Agree, 3 = Disagree, 4 = Strongly disagree

1     742
2    1444
3    1014
4     402
Name: S1MPERSON1, dtype: int64

Because variable names are not self-explanatory, attach variable description to each variable name.

In [16]:
# Run this after recoding

assert (df["X1SEX"].cat.codes.value_counts() == pd.Series([2006, 1596])).all()
df["X1SEX"].cat.codes.value_counts()

# Should be 0 = Female, 1 = Male

0    2006
1    1596
dtype: int64

In [17]:
# Run this after recoding

assert (
    df["S1MUNDERST"].cat.codes.value_counts().sort_index()
    == pd.Series([16, 141, 975, 2470])
).all()
df["S1MUNDERST"].cat.codes.value_counts().sort_index()

# Should be 0 = Never, 1 = Rarely, 2 = Sometimes, 3 = Often

0      16
1     141
2     975
3    2470
dtype: int64

In [18]:
# Run this after recoding

assert (
    df["S1MPERSON1"].cat.codes.value_counts().sort_index()
    == pd.Series([402, 1014, 1444, 742])
).all()
df["S1MPERSON1"].cat.codes.value_counts().sort_index()

# Should be 0 = Strongly disagree, 1 = Disagree, 2 = Agree, 3 = Strongly agree

0     402
1    1014
2    1444
3     742
dtype: int64

In [19]:
(
    df.assign(**{c: df[c].cat.codes for c in cat_columns})
    .pipe(lambda df: df[(df >= 0).apply(all, axis=1)])
    .to_csv("growth_analysis_data.csv", header=False, index=False)
)

In [20]:
num_columns = [
    "P1MTHCOMP",
    "P1SCICOMP",
    "S1MTCHMFDIFF",
    "S1STCHMFDIFF",
    "S1MTHCOMP",
    "S1SCICOMP",
    "X1TXMQUINT",
    "S1MPERSON1",
    "S2MPERSON1",
    "S4MPERSON1",
    "S1SPERSON1",
    "S2SPERSON1",
    "S4SPERSON1",
    "S1MUNDERST",
    "S1SUNDERST",
    "X5STEM1GPA",
]

**Female descriptives**

In [21]:
df[num_columns]

,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA
2,Males are much better,Males are somewhat better,Strongly disagree,Strongly disagree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Disagree,Strongly disagree,Strongly agree,Disagree,Strongly disagree,Strongly agree,Often,Sometimes,2.7
6,Females and males are the same,Females and males are the same,Disagree,Disagree,Females and males are the same,Females and males are the same,Third (middle) quintile,Agree,Agree,Agree,Strongly disagree,Strongly disagree,Agree,Often,Rarely,0.9
11,Females and males are the same,Females and males are the same,Strongly disagree,Strongly disagree,Males are somewhat better,Females and males are the same,Fifth (highest) quintile,Disagree,Strongly disagree,Disagree,Agree,Agree,Disagree,Often,Often,4.0
23,Males are somewhat better,Males are somewhat better,Disagree,Disagree,Females and males are the same,Males are somewhat better,Fifth (highest) quintile,Disagree,Disagree,Agree,Disagree,Agree,Agree,Often,Often,3.0
27,Females and males are the same,Females and males are the same,Disagree,Disagree,Males are somewhat better,Females and males are the same,Fifth (highest) quintile,Strongly disagree,Strongly disagree,Strongly disagree,Strongly disagree,Agree,Disagree,Sometimes,Sometimes,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23491,Females and males are the same,Females and males are the same,Strongly disagree,Disagree,Females and males are the same,Males are somewhat better,Fifth (highest) quintile,Strongly disagree,Disagree,Agree,Agree,Agree,Disagree,Sometimes,Sometimes,3.6
23494,Females and males are the same,Females and males are the same,Disagree,Disagree,Males are somewhat better,Females and males are the same,Fouth quintile,Agree,Agree,Agree,Strongly disagree,Disagree,Agree,Often,Sometimes,2.6
23495,Males are somewhat better,Females and males are the same,Strongly disagree,Strongly disagree,Males are somewhat better,Males are somewhat better,Fifth (highest) quintile,Strongly agree,Strongly agree,Agree,Strongly agree,Strongly agree,Strongly agree,Often,Often,3.3
23497,Females and males are the same,Females and males are the same,Disagree,Disagree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Agree,Strongly agree,Strongly agree,Agree,Strongly agree,Strongly agree,Often,Often,3.8


In [22]:
cdf = df.assign(**{c: df[c].cat.codes for c in cat_columns})

In [23]:
cdf.loc[cdf["X1SEX"] == 0][num_columns].describe()

,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA
count,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000,2006.000000
mean,2.163011,2.091226,0.620638,0.666500,1.898804,1.966102,2.849452,1.601196,1.466600,1.508475,1.444167,1.647059,1.701396,2.611167,2.463111,2.786092
std,0.759215,0.633364,0.648153,0.651064,0.764700,0.718718,1.211033,0.915632,1.021285,0.956238,0.843117,0.933745,0.901131,0.585234,0.653695,0.914162
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,0.000000,0.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.300000
50%,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,3.000000,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,3.000000,3.000000,3.000000
75%,3.000000,2.000000,1.000000,1.000000,2.000000,2.000000,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.500000
max,4.000000,4.000000,3.000000,3.000000,4.000000,4.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000


In [24]:
np.round(cdf.loc[cdf["X1SEX"] == 0][num_columns].corr(), 3)

,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA
P1MTHCOMP,1.000,0.332,0.008,-0.008,0.102,0.043,-0.031,-0.099,-0.072,-0.056,-0.023,0.006,-0.024,-0.055,-0.005,-0.011
P1SCICOMP,0.332,1.000,0.044,0.039,-0.000,0.057,0.024,-0.028,-0.010,0.019,-0.061,-0.071,-0.025,0.020,-0.032,0.055
S1MTCHMFDIFF,0.008,0.044,1.000,0.420,-0.020,-0.072,-0.031,-0.091,-0.067,-0.009,-0.035,-0.007,-0.030,-0.043,-0.029,-0.006
S1STCHMFDIFF,-0.008,0.039,0.420,1.000,-0.014,-0.025,-0.041,-0.066,-0.048,0.014,-0.106,-0.035,-0.017,-0.049,-0.082,0.008
S1MTHCOMP,0.102,-0.000,-0.020,-0.014,1.000,0.284,0.023,-0.152,-0.118,-0.111,0.055,0.023,-0.003,-0.081,0.049,0.061
S1SCICOMP,0.043,0.057,-0.072,-0.025,0.284,1.000,0.015,0.018,0.037,0.053,-0.152,-0.094,-0.085,0.067,-0.104,0.079
X1TXMQUINT,-0.031,0.024,-0.031,-0.041,0.023,0.015,1.000,0.299,0.297,0.263,0.192,0.166,0.149,0.290,0.233,0.354
S1MPERSON1,-0.099,-0.028,-0.091,-0.066,-0.152,0.018,0.299,1.000,0.558,0.463,0.173,0.102,0.139,0.482,0.071,0.117
S2MPERSON1,-0.072,-0.010,-0.067,-0.048,-0.118,0.037,0.297,0.558,1.000,0.581,0.090,0.165,0.174,0.330,0.029,0.179
S4MPERSON1,-0.056,0.019,-0.009,0.014,-0.111,0.053,0.263,0.463,0.581,1.000,0.059,0.089,0.219,0.281,0.044,0.198


**Male descriptives**

In [25]:
cdf.loc[cdf["X1SEX"] == 1][num_columns].describe()

,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA
count,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000,1596.000000
mean,2.406015,2.263784,0.722431,0.718045,2.182957,2.188596,3.032581,1.827068,1.765038,1.825188,1.666667,1.809524,1.832707,2.671053,2.560150,2.603759
std,0.720024,0.605071,0.762785,0.755801,0.862517,0.763687,1.133780,0.908432,0.978416,0.902879,0.885504,0.890245,0.847024,0.571147,0.630085,0.970530
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,0.000000,0.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
50%,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,2.800000
75%,3.000000,3.000000,1.000000,1.000000,3.000000,2.000000,4.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.300000
max,4.000000,4.000000,3.000000,3.000000,4.000000,4.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000


In [26]:
np.round(cdf.loc[cdf["X1SEX"] == 1][num_columns].corr(), 3)

,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA
P1MTHCOMP,1.000,0.302,0.022,0.048,0.125,0.040,0.075,0.119,0.130,0.090,-0.025,0.024,0.034,0.073,-0.025,0.067
P1SCICOMP,0.302,1.000,-0.002,0.020,0.059,0.047,0.010,-0.016,0.005,-0.013,0.031,0.039,0.073,-0.037,-0.018,0.015
S1MTCHMFDIFF,0.022,-0.002,1.000,0.423,-0.030,-0.003,-0.103,-0.127,-0.082,-0.069,-0.081,-0.025,-0.003,-0.095,-0.044,-0.042
S1STCHMFDIFF,0.048,0.020,0.423,1.000,0.043,0.014,-0.049,-0.016,0.001,-0.002,-0.066,0.001,-0.027,-0.029,-0.096,-0.025
S1MTHCOMP,0.125,0.059,-0.030,0.043,1.000,0.367,0.163,0.236,0.191,0.172,0.046,0.054,0.092,0.135,0.013,0.092
S1SCICOMP,0.040,0.047,-0.003,0.014,0.367,1.000,0.075,0.062,0.037,0.051,0.125,0.098,0.101,0.009,0.093,0.067
X1TXMQUINT,0.075,0.010,-0.103,-0.049,0.163,0.075,1.000,0.357,0.375,0.339,0.177,0.221,0.166,0.301,0.200,0.325
S1MPERSON1,0.119,-0.016,-0.127,-0.016,0.236,0.062,0.357,1.000,0.555,0.491,0.228,0.176,0.182,0.424,0.094,0.187
S2MPERSON1,0.130,0.005,-0.082,0.001,0.191,0.037,0.375,0.555,1.000,0.620,0.113,0.244,0.222,0.313,0.060,0.228
S4MPERSON1,0.090,-0.013,-0.069,-0.002,0.172,0.051,0.339,0.491,0.620,1.000,0.083,0.166,0.286,0.264,0.057,0.238


In [ ]:
col_na = round(100 * (df.isna().sum() / len(df)), 2)
col_nunique = df.nunique()
df_summary = pd.DataFrame(
    {
        "data_type": df.dtypes,
        "percent_missing_values": col_na,
        "total_unique_values": col_nunique,
    }
)
df_summary.pipe(codebook.relabel, axis=0)

In [ ]:
df = df.loc[df["X5STEM1GPA"] > 0]

In [ ]:
# df = df.loc[~df["P1MTHCOMP"].isnull()]

In [ ]:
# df = df.loc[~df["P1SCICOMP"].isnull()]

In [ ]:
# df = df.loc[~df["S4MPERSON1"].isnull()]

In [ ]:
# df = df.loc[~df["S4SPERSON1"].isnull()]

In [ ]:
# df = df.loc[~df["X1RACE"].isnull()] and df.loc[~df["X1SEX"].isnull()]

In [ ]:
df.dropna()

In [ ]:
df[["S1MPERSON1"]].value_counts()

## 1. Demographics

### 1.1 Frequencies and descriptives

In [ ]:
df["X1RACE"].value_counts(sort=False, dropna=False, normalize=True).pipe(
    codebook.relabel
).to_frame()

In [ ]:
df["X1WHITE"].value_counts(dropna=False, normalize=True).pipe(
    codebook.relabel
).to_frame()

Note: Not sure how the first category's count is 17,006, in relation to the frequencies for `X1RACE` (see above).

In [ ]:
df["X1SEX"].value_counts(dropna=False, normalize=True).pipe(codebook.relabel).to_frame()

In [ ]:
df["X1SESQ5"].value_counts(dropna=False, normalize=True).pipe(
    codebook.relabel
).to_frame()

In [ ]:
df["X3REGION"].value_counts(dropna=False, normalize=True).pipe(
    codebook.relabel
).to_frame()

In [ ]:
df["X1LOCALE"].value_counts(dropna=False, normalize=True).pipe(
    codebook.relabel
).to_frame()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
cat_columns = [
    "X1RACE",
    "X1WHITE",
    "X1SEX",
    "X1SESQ5",
    "X3REGION",
    "X1LOCALE",
    "P1MTHCOMP",
    "P1SCICOMP",
    "S1MTCHMFDIFF",
    "S1STCHMFDIFF",
    "S1MTHCOMP",
    "S1SCICOMP",
    "X1TXMQUINT",
    "S1MPERSON1",
    "S2MPERSON1",
    "S4MPERSON1",
    "S1SPERSON1",
    "S2SPERSON1",
    "S4SPERSON1",
    "S3FIELD_STEM",
    "X4ENTMJST",
]

In [ ]:
for column in cat_columns:
    pdf = df.pivot_table(index="X1SEX", columns=column, aggfunc="size", dropna=False)
    pdf = pdf.div(pdf.sum(axis=1), axis=0)
    g = pdf.pipe(codebook.relabel).plot.barh(stacked=True, figsize=(12, 6))
    g.legend(title=labels[column], loc="upper left", bbox_to_anchor=(0, -0.05), ncol=1)

Take a look at the 'math person' belief as a numeric variable.

In [ ]:
(df[cat_columns].dro.apply(lambda x: x.cat.codes))

In [ ]:
df[["S1MPERSON1"]].pipe(codebook.relabel).apply(lambda x: x.cat.codes)

In [ ]:
# .to_csv('growth_analysis_data.csv')